In [ ]:
import sys; sys.path.append('..')
import numpy as np, importlib
import mesh, fd_validation, parallelism

In [ ]:
np.random.seed(0)
m = mesh.Mesh('../../misc/examples/meshes/square_hole.off',  embeddingDimension=3)
m = mesh.Mesh('../../misc/examples/meshes/sphere_hires.msh', embeddingDimension=3)

In [ ]:
import elastic_sheet, energy, tensors, loads
et = tensors.ElasticityTensor2D(0.0, 0.3) # Young's modulus 0 disables the elastic energy term
psi = energy.StVenantKirchhoffCBased(et)
es = elastic_sheet.ElasticSheet(m, psi)

In [ ]:
gravity = loads.Gravity(es, 1.0)
spreaders = loads.Spreaders(es, [[0, 1], [2, 3]], [[0, 1]], 1)
springs = loads.Springs(es, [loads.AttachmentPointCoordinate([0], [1.0]), loads.AttachmentPointCoordinate([3, 6, 9], [1/3, 1/3, 1/3])], # Pull the x component of vertex 0, and the x component of the barycenter of vertices 2, 3, 4...
                            [loads.AttachmentPointCoordinate([3], [1.0]), loads.AttachmentPointCoordinate([], [0.0])],            # ... toward the x component of vertex 1, and the fixed value 0, respectively
                        1.0)

In [ ]:
prob = es.EquilibriumProblem([spreaders, gravity, springs])

In [ ]:
es.setVars(es.getVars() + 1e-1 * np.random.uniform(size=es.numVars()))

In [ ]:
fd_validation.gradConvergencePlot(prob)

In [ ]:
fd_validation.hessConvergencePlot(prob)